<a href="https://colab.research.google.com/github/sahar123456/introduction_to_keras/blob/master/Introduction_to_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
# Install the latest version of TensorFlow
!pip install -q -U tensorflow==1.12.0

     |████████████████████████████████| 83.1MB 42kB/s 
     |████████████████████████████████| 3.1MB 37.8MB/s 


In [3]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

# This code was tested with TensorFlow v1.12.0
print("You have TensorFlow version", tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

You have TensorFlow version 1.12.0


In [4]:
# Get the data: original source is here: https://www.kaggle.com/zynicide/wine-reviews/data
URL = "https://storage.googleapis.com/sara-cloud-ml/wine_data.csv"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)


49790976/49783940 [==============================] - 1s 0us/step


In [0]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

In [8]:
# Shuffle the data
data = data.sample(frac=1)

# Print the first 5 rows
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
67262,67262,France,"A fascinating nose, with grapefruit, kiwi, cit...",Cuvée Henri,88,30.0,Alsace,Alsace,NaN,Riesling,Lucien Albrecht
48190,48190,US,Stainless steel-fermented and kept on the lees...,NaN,86,15.0,Oregon,Willamette Valley,Willamette Valley,Pinot Gris,Walnut City WineWorks
68401,68401,South Africa,A strong nose of cat pee and green bean leads ...,NaN,85,15.0,Durbanville,NaN,NaN,Sauvignon Blanc,Nitída
70617,70617,Germany,"Serious and earthy, like walking into a wall o...",Graacher Himmelreich Spätlese,91,NaN,Mosel,NaN,NaN,Riesling,Joh. Jos. Prüm
50623,50623,Spain,Lively right off the bat but also sort of jumb...,Abadal,87,18.0,Catalonia,Pla de Bages,NaN,Cabernet Franc-Tempranillo,Masies d'Avinyo


In [0]:
# Do some preprocessing to limit the # of wine varities in the dataset
data = data[pd.notnull(data['country'])]
data = data[pd.notnull(data['price'])]
data = data.drop(data.columns[0], axis=1) 

variety_threshold = 500 # Anything that occurs less than this will be removed.
value_counts = data['variety'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['variety'])]

In [11]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 95646
Test size: 23912


In [0]:
# Train features
description_train = data['description'][:train_size]
variety_train = data['variety'][:train_size]

# Train labels
labels_train = data['price'][:train_size]

# Test features
description_test = data['description'][train_size:]
variety_test = data['variety'][train_size:]

# Test labels
labels_test = data['price'][train_size:]

In [0]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 12000 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) # only fit on train

In [0]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector 
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

In [20]:
#Wide feature 2: one-hot vector of variety categories
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(variety_train)
variety_train = encoder.transform(variety_train)
variety_test = encoder.transform(variety_test)
num_classes = np.max(variety_train) + 1

# Convert labels to one hot
variety_train = keras.utils.to_categorical(variety_train, num_classes)
variety_test = keras.utils.to_categorical(variety_test, num_classes)

ValueError: ignored

In [0]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)

In [22]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12000)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12040)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          3082496     concatenate[0][0]                
__________

In [0]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [24]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 170)               0         
_________________________________________________________________
embedding (Embedding)        (None, 170, 8)            96000     
_________________________________________________________________
flatten (Flatten)            (None, 1360)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1361      
Total params: 97,361
Trainable params: 97,361
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
deep_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

In [26]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12000)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 170)          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12040)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________

In [27]:
# Run training
combined_model.fit([description_bow_train, variety_train] + [train_embed], labels_train, epochs=10, batch_size=128)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
95646/95646 [==============================] - 75s 784us/step - loss: 1171.4533 - acc: 0.0000e+00
Epoch 2/10
95646/95646 [==============================] - 73s 764us/step - loss: 917.6726 - acc: 0.0000e+00
Epoch 3/10
95646/95646 [==============================] - 73s 763us/step - loss: 770.0032 - acc: 0.0000e+00
Epoch 4/10
95646/95646 [==============================] - 73s 766us/step - loss: 640.1262 - acc: 0.0000e+00
Epoch 5/10
95646/95646 [==============================] - 73s 768us/step - loss: 518.7716 - acc: 0.0000e+00
Epoch 6/10
95646/95646 [==============================] - 74s 770us/step - loss: 405.8370 - acc: 0.0000e+00
Epoch 7/10
95646/95646 [==============================] - 73s 764us/step - loss: 313.1239 - acc: 0.0000e+00
Epoch 8/10
95646/95646 [==============================] - 74s 770us/step - loss: 237.8823 - acc: 0.0000e+00
Epoch 9/10
95646/95646 [==============================] - 74s 771us/step - loss: 181.1562 - acc: 0.0000e+00
Epoch 10/10
95646/95646 [==

In [28]:
combined_model.evaluate([description_bow_test, variety_test] + [test_embed], labels_test, batch_size=128)

23912/23912 [==============================] - 7s 287us/step


[536.3136072704408, 0.0]

In [0]:
# Generate predictions
predictions = combined_model.predict([description_bow_test, variety_test] + [test_embed])

In [32]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

Soft, rich, but still with fine acidity to keep the freshness, this is a white fruit- and citrus-flavored wine that broadens well in the mouth. The fruit is ripe, hinting at almonds, with just a light toast edge. Imported by Wilson Daniels Ltd.
Predicted:  46.355 Actual:  29.0 

Cabernet, Merlot, Sangiovese and Syrah produce a sophisticated and refined blend that boasts fantastic power and intensity. The wine is dark, chewy, rich and bursting with flavors of mature berry, spice, cola, leather and tobacco.
Predicted:  32.490738 Actual:  35.0 

There's lots of rich acidity in this wine. It braces the tremendous pear and pineapple fruit and sweet oak, and lends the wine zest and structure. Finishes with a brilliant array of tangy spices.
Predicted:  23.361502 Actual:  28.0 

Sommeliers, run, don't walk, to scoop this one up by the case. It's really good, a dry, softly balanced Cab that will back up food, not dominate it. Ready to drink now for its easy tannins and upfront blackberry and c

In [33]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

Average prediction difference:  15.442932295799256
